In [1]:
import os
os.environ['http_proxy'] = 'http://proxygate2.ctripcorp.com:8080'
os.environ['https_proxy'] = 'http://proxygate2.ctripcorp.com:8080'
CUDA_VISIBLE_DEVICES=1

In [2]:
# Adapted from Hugging Face tutorial: https://huggingface.co/docs/transformers/training

import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)

# Datasets
dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = dataset["train"].select(range(1000)).map(tokenize_function, batched=True)
small_eval_dataset = dataset["test"].select(range(1000)).map(tokenize_function, batched=True)

# Model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

# Metrics
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Hugging Face Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", report_to="none")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# Start Training
trainer.train()

[2023-10-30 07:50:36,485] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/envs/zzd-env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.455955,0.372000
2,No log,1.095957,0.512000
3,No log,1.055615,0.525000


/root/miniconda3/envs/zzd-env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/root/miniconda3/envs/zzd-env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=189, training_loss=1.2343917120070684, metrics={'train_runtime': 102.3268, 'train_samples_per_second': 29.318, 'train_steps_per_second': 1.847, 'total_flos': 789354427392000.0, 'train_loss': 1.2343917120070684, 'epoch': 3.0})

In [2]:
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)

import ray.train.huggingface.transformers
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

# [1] Encapsulate data preprocessing, training, and evaluation
# logic in a training function
def train_func(config):
    # Datasets
    dataset = load_dataset("yelp_review_full")
    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    small_train_dataset = dataset["train"].select(range(1000)).map(tokenize_function, batched=True)
    small_eval_dataset = dataset["test"].select(range(1000)).map(tokenize_function, batched=True)

    # Model
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

    # Evaluation Metrics
    metric = evaluate.load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    # Hugging Face Trainer
    training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", report_to="none")

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
    )

    # [2] Report Metrics and Checkpoints to Ray Train
    callback = ray.train.huggingface.transformers.RayTrainReportCallback()
    trainer.add_callback(callback)

    # [3] Prepare Transformers Trainer
    trainer = ray.train.huggingface.transformers.prepare_trainer(trainer)

    # Start Training
    trainer.train()

# [4] Define a Ray TorchTrainer to launch `train_func` on all workers
ray_trainer = TorchTrainer(train_func, scaling_config=ScalingConfig(num_workers=2, use_gpu=True))
ray_trainer.fit()

(TrainTrainable pid=756917) [2023-10-30 08:02:57,057] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


(TorchTrainer pid=756917) Starting distributed worker processes: ['757221 (10.60.134.127)', '757222 (10.60.134.127)']
(RayTrainWorker pid=757221) Setting up process group for: env:// [rank=0, world_size=2]


(RayTrainWorker pid=757221) [2023-10-30 08:03:08,085] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
(RayTrainWorker pid=757222) [2023-10-30 08:03:08,124] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Map: 100%|██████████| 1000/1000 [00:00<00:00, 2944.89 examples/s]
(RayTrainWorker pid=757221) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
(RayTrainWorker pid=757221) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1000/1000 [00:00<00:00, 2458.23 examples/s]
(RayTrainWorker pid=757222) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
(RayTrainWorker pid=757222) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(RayTrainWorker pid=757221) [W reducer.cpp:1300] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward 

(RayTrainWorker pid=757221) {'eval_loss': 1.280670166015625, 'eval_accuracy': 0.404, 'eval_runtime': 6.0052, 'eval_samples_per_second': 166.524, 'eval_steps_per_second': 10.491, 'epoch': 1.0}


...

(RayTrainWorker pid=757221) {'eval_loss': 1.0747922658920288, 'eval_accuracy': 0.52, 'eval_runtime': 6.0137, 'eval_samples_per_second': 166.287, 'eval_steps_per_second': 10.476, 'epoch': 2.0}


(RayTrainWorker pid=757221) 
                                                 A
100%|██████████| 63/63 [00:05<00:00, 10.82it/s]
                                               
100%|██████████| 189/189 [01:09<00:00,  3.32it/s]
(RayTrainWorker pid=757221) 
  0%|          | 0/63 [00:00<?, ?it/s]
(RayTrainWorker pid=757221) 
  5%|▍         | 3/63 [00:00<00:03, 15.95it/s]
(RayTrainWorker pid=757221) 
  8%|▊         | 5/63 [00:00<00:04, 12.84it/s]
(RayTrainWorker pid=757221) 
 11%|█         | 7/63 [00:00<00:04, 11.84it/s]
(RayTrainWorker pid=757221) 
 14%|█▍        | 9/63 [00:00<00:04, 11.31it/s]
(RayTrainWorker pid=757221) 
 17%|█▋        | 11/63 [00:00<00:04, 11.04it/s]
(RayTrainWorker pid=757221) 
 21%|██        | 13/63 [00:01<00:04, 10.80it/s]
(RayTrainWorker pid=757221) 
 24%|██▍       | 15/63 [00:01<00:04, 10.76it/s]
(RayTrainWorker pid=757221) 
 27%|██▋       | 17/63 [00:01<00:04, 10.73it/s]
(RayTrainWorker pid=757221) 
 30%|███       | 19/63 [00:01<00:04, 10.70it/s]
(RayTrainWorker p

(RayTrainWorker pid=757221) {'eval_loss': 1.0825996398925781, 'eval_accuracy': 0.531, 'eval_runtime': 6.0244, 'eval_samples_per_second': 165.991, 'eval_steps_per_second': 10.457, 'epoch': 3.0}
(RayTrainWorker pid=757221) {'train_runtime': 75.567, 'train_samples_per_second': 39.7, 'train_steps_per_second': 2.501, 'train_loss': 1.125707636434565, 'epoch': 3.0}
Trial TorchTrainer_b97a0_00000 completed. Last result: 


2023-10-30 08:04:44,721	INFO tune.py:1143 -- Total run time: 112.65 seconds (112.62 seconds for the tuning loop).


Result(
  metrics={},
  path='/root/ray_results/TorchTrainer_2023-10-30_08-02-48/TorchTrainer_b97a0_00000_0_2023-10-30_08-02-52',
  filesystem='local',
  checkpoint=None
)